In [21]:
import pandas as pd
import numpy as np
import geopandas as gpd

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [22]:
df = pd.read_json("../scripts/Metropolis_CVE.json")

In [47]:
df.loc["mpos",'ZMVM']

['09002',
 '09003',
 '09004',
 '09005',
 '09006',
 '09007',
 '09008',
 '09009',
 '09010',
 '09011',
 '09012',
 '09013',
 '09014',
 '09015',
 '09016',
 '09017',
 '13069',
 '15002',
 '15009',
 '15010',
 '15011',
 '15013',
 '15015',
 '15016',
 '15017',
 '15020',
 '15022',
 '15023',
 '15024',
 '15025',
 '15028',
 '15029',
 '15030',
 '15031',
 '15033',
 '15034',
 '15035',
 '15036',
 '15037',
 '15038',
 '15039',
 '15044',
 '15046',
 '15050',
 '15053',
 '15057',
 '15058',
 '15059',
 '15060',
 '15061',
 '15065',
 '15068',
 '15069',
 '15070',
 '15075',
 '15081',
 '15083',
 '15084',
 '15089',
 '15091',
 '15092',
 '15093',
 '15094',
 '15096',
 '15099',
 '15100',
 '15103',
 '15104',
 '15108',
 '15109',
 '15112',
 '15120',
 '15121',
 '15122',
 '15125']

In [36]:
mun_gdf = gpd.GeoDataFrame()
mpos_folder = 'mpos_2020'
c = 'ZMVM'
# Iterates over municipality codes for each metropolitan area or capital
for i in range(len(df.loc["mpos", c])):
    # Extracts specific municipality code
    m = df.loc["mpos", c][i]
    # Downloads municipality polygon according to code
    query = f"SELECT * FROM marco.{mpos_folder} WHERE \"CVEGEO\" LIKE \'{m}\'"
    mun_gdf = mun_gdf.append(aup.gdf_from_query(query, geometry_col='geometry'))

In [37]:
gdf_tmp = mun_gdf.copy()
gdf_tmp = gdf_tmp.to_crs("EPSG:6372")
gdf_tmp = gdf_tmp.buffer(1).reset_index().rename(columns={0:'geometry'})
gdf_tmp = gdf_tmp.to_crs("EPSG:4326")
poly_wkt = gdf_tmp.dissolve().geometry.to_wkt()[0]

In [38]:
query = f"SELECT * FROM censoageb.censoageb_2020 WHERE (ST_Intersects(geometry, \'SRID=4326;{poly_wkt}\'))"
ageb = aup.gdf_from_query(query, geometry_col='geometry')

In [39]:
print(ageb.shape)
ageb.head(2)

(5855, 237)


,cve_geo,cve_ent,cve_mun,cve_loc,cve_ageb,geometry,entidad,nom_ent,mun,nom_mun,...,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sintlc,vph_sincint,vph_sintic,cve_geo_ageb
0,0901000011716,09,010,0001,1716,"POLYGON ((-99.25882 19.32558, -99.25834 19.325...",9,Ciudad de México,10,Álvaro Obregón,...,1660.0,1181.0,644.0,279.0,210.0,45.0,82.0,601.0,9.0,0901000011716
1,0901000011307,09,010,0001,1307,"POLYGON ((-99.20805 19.31277, -99.20768 19.312...",9,Ciudad de México,10,Álvaro Obregón,...,839.0,831.0,768.0,640.0,348.0,NaN,0.0,18.0,0.0,0901000011307


In [40]:
ageb = ageb.to_crs("EPSG:6372")

In [52]:
ageb['area'] = ageb.area

In [53]:
((ageb['area']/10000)/100).sum()

2347.778924327681

In [54]:
ageb.pobtot.sum()

21056688

In [41]:
ageb = ageb[['cve_geo','geometry']].copy()

In [19]:
ageb.dissolve()

,geometry,cve_geo
0,"MULTIPOLYGON (((-116.91632 32.21281, -116.9168...",0200303401550


In [42]:
ageb.dissolve().to_file(f'/home/edgar/SIG/ObsCds/Proyectos/Guggenheim/Medio_Urbano/Mancha_Urbana/AGEB2020_{c}.geojson')